In [44]:
import os
import torch
import torch.nn as nn 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU index

print("Available GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print(device)

Available GPUs: 1
GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
cuda:0


In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import torch 
import sys
sys.path.insert(1, os.path.join("..", "data"))
sys.path.insert(1, os.path.join("..", "utils"))
from data_utils import Dataset
from plot_utils import plot_image
from torch.utils.data import DataLoader

In [47]:
# Path to the data folder (update the variable to your path).
path_data=os.path.join("..", "data")
# Seed value
seed=1001

In [48]:
dataset=Dataset(path_data=path_data, seed=seed)
dataset.read_data()
dataset.get_statistics()

Parsing class: Cloud: 143it [00:07, 19.06it/s]
Parsing class: Edge: 97it [00:04, 21.92it/s]
Parsing class: Good: 64it [00:03, 16.82it/s]


,train,valid,test
cloud,100,24,19
edge,64,15,18
good,48,7,9


In [49]:
batch_size=32
# Train loader
train_loader = DataLoader(dataset.get_split("train"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Cross validation data loader
valid_loader = DataLoader(dataset.get_split("valid"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Test data loader
test_loader = DataLoader(dataset.get_split("test"), batch_size=batch_size, pin_memory=False, shuffle=True)

In [50]:
classes = ('cloud', 'edge', 'good')

Start of training loop

In [51]:
import torch.nn.functional as F 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def resize_tensor_images(images, size=(256, 256)):
    # Resize the batch of images
    return F.interpolate(images, size=size, mode='bilinear', align_corners=False)

def compute_mean_std(loader):
    # Computation of mean and standard deviation of batches
    mean = 0.
    std = 0.
    total_images_count = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count

    return mean, std

def normalize_images(images, mean, std):
    # Normalizing images with previously computed mean and standard deviation
    normalized_images = (images - mean.view(-1, 1, 1)) / std.view(-1, 1, 1)
    return normalized_images
    
def tensor_to_numpy(tensor):
    # Rescale the tensor to 0-1 range
    tensor = tensor - tensor.min()
    tensor = tensor / tensor.max()
    # Move the tensor to CPU if it's on GPU
    tensor = tensor.cpu()
    # Convert to numpy and transpose from CxHxW to HxWxC for visualization
    numpy_image = tensor.numpy()
    numpy_image = np.transpose(numpy_image, (1, 2, 0))

    return numpy_image

In [52]:
mean, std = compute_mean_std(test_loader)

In [53]:
def normalization(data_loader, mean, std):
    UNPRO_batches = []
    batches = []
    
    for batch in data_loader:
        images, labels = batch
        resized_images = resize_tensor_images(images)
        UNPRO_batches.append((resized_images, labels))
        normalized_alldata_images = normalize_images(resized_images, mean, std)

        # Append the normalized images and their corresponding labels to the list
        batches.append((normalized_alldata_images, labels))
    return UNPRO_batches, batches

UNPRO_batches_TRL, batches_TRL = normalization(train_loader, mean, std)
UNPRO_batches_VAL, batches_VAL = normalization(valid_loader, mean, std)
UNPRO_batches_TST, batches_TST = normalization(test_loader, mean, std)


In [54]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
            nn.Conv2d(3, 25, kernel_size=3, padding=1),
            nn.BatchNorm2d(25),
            nn.ReLU(),
            nn.Conv2d(25, 25, kernel_size=3, padding=1),
            nn.ReLU(),
        )
        self.conv_layer_2 = nn.Sequential(
            nn.Conv2d(25, 25, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(25, 25, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=409600,
                      out_features=3)
        )
    def forward(self, x):

        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.classifier(x)

        return x
    
net = Net().to(device)

In [55]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [56]:
from torchmetrics import Accuracy
accuracy_fn = Accuracy(task="multiclass", num_classes=3).to(device)

In [57]:
def train_step(model: torch.nn.Module,
               batches,
               loss_fn,
               optimizer,
               accuracy,
               device: torch.device = device):
    model.train()
    train_loss, train_acc = 0, 0

    for batch, (images, labels) in enumerate(batches, 0):
        
        images, labels = images.to(device), labels.to(device)

        y_logits = model(images)

        # Calculate loss on 1 batch of data
        loss = loss_fn(y_logits, labels)
        acc = accuracy(y_logits.argmax(dim=-1), labels)
        train_loss += loss
        train_acc += acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_acc = train_acc / len(batches)
    train_loss = train_loss / len(batches)

    print(f"Training Accuracy: {train_acc*100:.3f}, Train Loss: {train_loss:.3f}")
    return train_acc, train_loss

In [61]:
def validation_step(model,
              batches,
              loss_fn,
              accuracy,
              device: torch.device = device):
    validation_loss, validation_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for images_validation, labels_validation in batches:
            images_validation, labels_validation = images_validation.to(device), labels_validation.to(device)
            
            validation_logits = model(images_validation)
            validation_loss += loss_fn(validation_logits, labels_validation)
            validation_acc += accuracy(validation_logits.argmax(dim=-1), labels_validation)

        validation_loss /= len(batches)
        validation_acc /= len(batches)
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_acc*100:.4f}%")
    return validation_acc, validation_loss

In [ ]:
def test_step(model,
              batches,
              loss_fn,
              accuracy,
              device: torch.device = device):
    validation_loss, validation_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for images_validation, labels_validation in batches:
            images_validation, labels_validation = images_validation.to(device), labels_validation.to(device)
            
            validation_logits = model(images_validation)
            validation_loss += loss_fn(validation_logits, labels_validation)
            validation_acc += accuracy(validation_logits.argmax(dim=-1), labels_validation)

        validation_loss /= len(batches)
        validation_acc /= len(batches)
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_acc*100:.4f}%")
    return validation_acc, validation_loss

In [60]:

epochs = 20

for epoch in range(epochs):
    train_step(model=net,
               batches=batches_TRL,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy=accuracy_fn,
               device=device
               )
    validation_step(model=net,
              batches=batches_VAL,
              loss_fn=loss_fn,
              accuracy=accuracy_fn,
              device=device)
    validation_step(model=net,
              batches=batches_TST,
              loss_fn=loss_fn,
              accuracy=accuracy_fn,
              device=device)

Training Accuracy: 62.768, Train Loss: 0.851
Validation Loss: 0.8404, Validation Accuracy: 70.0893%
Validation Loss: 0.8481, Validation Accuracy: 62.2768%
Training Accuracy: 63.214, Train Loss: 0.832
Validation Loss: 0.8296, Validation Accuracy: 70.0893%
Validation Loss: 0.8359, Validation Accuracy: 62.2768%
Training Accuracy: 63.214, Train Loss: 0.813
Validation Loss: 0.8201, Validation Accuracy: 70.0893%
Validation Loss: 0.8236, Validation Accuracy: 63.8393%
Training Accuracy: 63.214, Train Loss: 0.794
Validation Loss: 0.8125, Validation Accuracy: 70.0893%
Validation Loss: 0.8119, Validation Accuracy: 63.8393%
Training Accuracy: 64.375, Train Loss: 0.774
Validation Loss: 0.8037, Validation Accuracy: 71.6518%
Validation Loss: 0.8000, Validation Accuracy: 65.4018%
Training Accuracy: 64.375, Train Loss: 0.755
Validation Loss: 0.7942, Validation Accuracy: 71.6518%
Validation Loss: 0.7880, Validation Accuracy: 66.9643%
Training Accuracy: 66.964, Train Loss: 0.736
Validation Loss: 0.7861, 